In [61]:
import numpy as np
import pandas as pd
from func import *

np.set_printoptions(precision=3, suppress=True)

In [62]:
#inf = np.finfo(np.float32).max
inf = 1_000_000

In [63]:
C = np.array([
    [0, 0],
    [3, 4],
    [9, 4],
    [0, 8],
    [9, 8]
])
assert np.shape(C)[1] == 2, "Las coordenadas se componen de 2 valores."

In [64]:
S = np.array([
    [0,0,0],
    [1,1,0],
    [1,1,1],
    [0,0,0],
    [0,0,0]

])
assert np.shape(S)[1] == 3, "La matriz S debe estar compuesta de 3 columnas."
assert np.shape(S)[0] == np.shape(C)[0], "La matriz S y la matriz C deben tener la misma forma."

In [65]:
B = np.array([
    [0,1],
    [1,2],
    [1,3],
    [2,4],
])

In [66]:
EAI = np.array([
    [1, inf, 0.00000001],
    [1, inf, 0.00000001],
    [1, 1, inf],
    [1, inf, 0.00000001],
])

In [67]:
L = np.linalg.norm(np.diff(C[B], axis=1), axis=2)
L

array([[5.],
       [6.],
       [5.],
       [4.]])

In [68]:
with np.errstate(divide='ignore'):
    alpha = np.arctan(np.divide(*(np.diff(C[B], axis=1).squeeze().T[::-1])))
np.rad2deg(alpha)

array([ 53.13,   0.  , -53.13,  90.  ])

In [69]:
EAIL = np.hstack([EAI, L])

In [70]:
with np.errstate(divide='ignore', ):    
    Ki = []
    A = []
    Kt = []
    for el, t in zip(EAIL, alpha):
        Ki.append(stiffness_2d(*el))
        A.append(rot_2d(t))
        Kt.append(A[-1].T @ Ki[-1] @ A[-1])
    Ki = np.array(Ki)
    A = np.array(A)
    Kt = np.array(Kt)

In [71]:
def format(value) -> str:
    #if np.abs(value) > 10000:
    #    return 'inf'
    return value.format

pd.options.display.float_format = '{:.3F}'.format

In [72]:
Kg = np.zeros(shape=(S.size,S.size))
GLg = np.zeros(shape=(S.size, 1))
_idx = np.arange(0, S.size)
G_idx = _idx.reshape(len(S), -1)[B].reshape(-1, 6)
for idx, Ki, GL in zip(G_idx, Kt, S[B]):
    Kg[*np.meshgrid(idx, idx)] += Ki
    # GLg[idx] += GL.ravel()
GLg = S.ravel()
Kg.shape, GLg.shape

((15, 15), (15,))

In [73]:
cols = _idx.reshape(S.shape)[B].reshape(-1, 6)


In [74]:
show = pd.DataFrame(Kt[0])
show

,0,1,2,3,4,5
0,72000.000,96000.000,-0.000,-72000.000,-96000.000,-0.000
1,96000.000,128000.000,0.000,-96000.000,-128000.000,0.000
2,-0.000,0.000,0.000,0.000,-0.000,0.000
3,-72000.000,-96000.000,0.000,72000.000,96000.000,0.000
4,-96000.000,-128000.000,-0.000,96000.000,128000.000,-0.000
5,-0.000,0.000,0.000,0.000,-0.000,0.000


In [75]:
Kg.shape

(15, 15)

In [76]:
pd.DataFrame(Kg, index=_idx + 1, columns=_idx + 1)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,72000.000,96000.000,-0.000,-72000.000,-96000.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2,96000.000,128000.000,0.000,-96000.000,-128000.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,-0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,-72000.000,-96000.000,0.000,300106.739,142079.904,192000.000,-166666.667,0.000,0.000,-61440.072,-46079.904,192000.000,0.000,0.000,0.000
5,-96000.000,-128000.000,-0.000,142079.904,162560.128,144000.000,0.000,-0.000,0.000,-46079.904,-34560.128,144000.000,0.000,0.000,0.000
6,-0.000,0.000,0.000,192000.000,144000.000,800000.000,0.000,-0.000,0.000,-192000.000,-144000.000,400000.000,0.000,0.000,0.000
7,0.000,0.000,0.000,-166666.667,0.000,0.000,166666.667,0.000,-0.000,0.000,0.000,0.000,-0.000,-0.000,-0.000
8,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,250000.000,-0.000,0.000,0.000,0.000,-0.000,-250000.000,0.000
9,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000
10,0.000,0.000,0.000,-61440.072,-46079.904,-192000.000,0.000,0.000,0.000,61440.072,46079.904,-192000.000,0.000,0.000,0.000


In [78]:
pd.DataFrame(np.diag(GLg) @ Kg @ np.diag(GLg), index=_idx + 1, columns=_idx + 1)\
    .pipe(lambda x: x.loc[(x != 0).any(axis=0), (x != 0).any(axis=1)])

,4,5,7,8,9
4,300106.739,142079.904,-166666.667,0.000,0.000
5,142079.904,162560.128,0.000,-0.000,0.000
7,-166666.667,0.000,166666.667,0.000,-0.000
8,0.000,-0.000,0.000,250000.000,-0.000
9,0.000,0.000,-0.000,-0.000,0.000
